# March Madness Kaggle 2022 Competition Code

In [76]:
#imports
import os
import re
import sklearn
import numpy as np 
import pandas as pd
import opendatasets as od

In [77]:
# bringing the data file into my notebook
od.download('https://www.kaggle.com/c/mens-march-mania-2022/data?select=MDataFiles_Stage2',force=True)

100%|██████████| 45.8M/45.8M [00:00<00:00, 137MB/s] 



Extracting archive ./mens-march-mania-2022/mens-march-mania-2022.zip to ./mens-march-mania-2022


In [78]:
# load in data
data_path = 'MDataFiles_Stage2/'
seeds = pd.read_csv(data_path + 'MNCAATourneySlots.csv') # tournament games
regular_results = pd.read_csv(data_path + 'MRegularSeasonDetailedResults.csv') # results of regular season X
tour_results = pd.read_csv(data_path + 'MNCAATourneyDetailedResults.csv') # results of tournament Y
submission = pd.read_csv(data_path + 'MSampleSubmissionStage2.csv') # test
rankings = pd.read_csv(data_path + 'MMasseyOrdinals_thruDay128.csv')
names = pd.read_csv(data_path +'MTeams.csv') # names

# Baseline Model
I first wanted to make a basic model that would use the average ranking of the team on the 128th day (right before the tournament) to make my prediction.

In [79]:
# how i will make prediction based on the average rank of both teams
# abstract function i came up with and can be optimized
def sigmoid(A,B):
    value = 1/(1+2**((A-B)/10))
    return round(value,2)

In [80]:
#results of each tournament game
def win(w,l):
    return (w<l)*1
train = tour_results.copy()[['Season','WTeamID','LTeamID']]
train['Outcome'] = train.apply(lambda x: win(x['WTeamID'], x['LTeamID']), axis=1)
train.head(5)

,Season,WTeamID,LTeamID,Outcome
0,2003,1421,1411,0
1,2003,1112,1436,1
2,2003,1113,1272,1
3,2003,1141,1166,1
4,2003,1143,1301,1


In [81]:
last_day = rankings[rankings['RankingDayNum'] == 128]
rank = last_day.groupby(['Season','TeamID']).mean()
rank.reset_index(inplace = True)
rank.head(5)

,Season,TeamID,RankingDayNum,OrdinalRank
0,2003,1102,128.0,170.000000
1,2003,1103,128.0,171.666667
2,2003,1104,128.0,39.000000
3,2003,1105,128.0,301.666667
4,2003,1106,128.0,224.666667


In [82]:
# Create data frame with everything we need to access how well our baseline model did on past tournaments
temp = (pd.merge(train, rank,  how='left', left_on=['Season','WTeamID'], right_on = ['Season','TeamID'])
          .drop(['RankingDayNum','TeamID'],axis = 1).rename(columns = {'OrdinalRank':'rankW'}))
final_df = (pd.merge(temp, rank,  how='left', left_on=['Season','LTeamID'], right_on = ['Season','TeamID'])
          .drop(['RankingDayNum','TeamID'],axis = 1).rename(columns = {'OrdinalRank':'rankL'}))

final_df['pred'] = final_df.apply(lambda x: sigmoid(x['rankW'], x['rankL']), axis=1)
final_df.head()

,Season,WTeamID,LTeamID,Outcome,rankW,rankL,pred
0,2003,1421,1411,0,230.000000,211.000000,0.21
1,2003,1112,1436,1,1.666667,133.666667,1.00
2,2003,1113,1272,1,38.666667,26.333333,0.30
3,2003,1141,1166,1,57.666667,17.333333,0.06
4,2003,1143,1301,1,34.666667,70.000000,0.92


In [83]:
from sklearn.metrics import log_loss
loss = log_loss(final_df['Outcome'],final_df['pred'])
loss

4.681288047546632

Obviously the log loss from our scores is very bad as I used a very random sigmoid function and only the rank of the teams on the last day to make a prediction

In [84]:
from sklearn.metrics import accuracy_score
final_df['pred_Outcome'] = final_df['pred'].round()
acc = accuracy_score(final_df['Outcome'],final_df['pred_Outcome'])
acc

0.49195596951735815

This strategy resulted in an accuracy which is about as good as guessing, but lets make a submission just for fun, using this strategy, you never know.

In [85]:
# new df with season and teams in own column
sub = submission.copy()
sub['Season'] = sub['ID'].apply(lambda x: int(x.split('_')[0]))
sub['TeamIDA'] = sub['ID'].apply(lambda x: int(x.split('_')[1]))
sub['TeamIDB'] = sub['ID'].apply(lambda x: int(x.split('_')[2])) 
sub.head(5)

,ID,Pred,Season,TeamIDA,TeamIDB
0,2022_1103_1104,0.5,2022,1103,1104
1,2022_1103_1112,0.5,2022,1103,1112
2,2022_1103_1116,0.5,2022,1103,1116
3,2022_1103_1120,0.5,2022,1103,1120
4,2022_1103_1124,0.5,2022,1103,1124


In [86]:
# only 2022 season
last_day = rankings[rankings['RankingDayNum'] == 128]
rank2022 = last_day[last_day['Season'] == 2022].groupby('TeamID').mean()['OrdinalRank']
rank2022 = rank2022.rename_axis('TeamIDA')
sub = sub.join(rank2022, on = 'TeamIDA', how = 'left').rename(columns = {'OrdinalRank':'rankA'})
rank2022 = rank2022.rename_axis('TeamIDB')
sub = sub.join(rank2022, on = 'TeamIDB', how = 'left').rename(columns = {'OrdinalRank':'rankB'})
sub['Pred'] = sub.apply(lambda x: sigmoid(x['rankA'], x['rankB']), axis=1)
sub.loc[5:10]

,ID,Pred,Season,TeamIDA,TeamIDB,rankA,rankB
5,2022_1103_1129,0.00,2022,1103,1129,137.035088,34.070175
6,2022_1103_1136,0.97,2022,1103,1136,137.035088,187.842105
7,2022_1103_1151,0.01,2022,1103,1151,137.035088,73.649123
8,2022_1103_1159,0.54,2022,1103,1159,137.035088,139.333333
9,2022_1103_1161,0.00,2022,1103,1161,137.035088,29.966667
10,2022_1103_1163,0.00,2022,1103,1163,137.035088,19.866667


In [87]:
sub[['ID','Pred']].to_csv('Submission',index = False)

# Logistic Regression and Other Classifiers
now lets make a predition using a little bit more advanced methods and more data

In [88]:
# the box scores of every game played during the regualar season separeted into 2 tables based on how won
cols_drop_loser = ['DayNum','NumOT']
cols_drop_winner = ['DayNum','NumOT']
for x in regular_results.columns:
    if x[0] == 'W':
        cols_drop_loser.append(x)
    if x[0] == 'L':
        cols_drop_winner.append(x)
cols_drop_loser.remove('WScore')
cols_drop_winner.remove('LScore')
loser = regular_results.drop(cols_drop_loser,axis =1)
winner = regular_results.drop(cols_drop_winner,axis =1)
loser['loses'] = 1
loser['wins'] = 0
winner["wins"] = 1
winner['loses'] = 0
winner.head()

,Season,WTeamID,WScore,LScore,WLoc,WFGM,WFGA,WFGM3,WFGA3,WFTM,WFTA,WOR,WDR,WAst,WTO,WStl,WBlk,WPF,wins,loses
0,2003,1104,68,62,N,27,58,3,14,11,18,14,24,13,23,7,1,22,1,0
1,2003,1272,70,63,N,26,62,8,20,10,19,15,28,16,13,4,4,18,1,0
2,2003,1266,73,61,N,24,58,8,18,17,29,17,26,15,10,5,2,25,1,0
3,2003,1296,56,50,N,18,38,3,9,17,31,6,19,11,12,14,2,18,1,0
4,2003,1400,77,71,N,30,61,6,14,11,13,17,22,12,14,4,4,20,1,0


In [89]:
# getting dictionaries to convert the column names so the loser and winner table can be combined
wdict = {}
for x in winner.columns:
    if x[0] == 'W':
        wdict[x] = x[1:]
ldict = {}
for x in loser.columns:
    if x[0] == 'L':
        ldict[x] = x[1:]
        
winner = winner.rename(columns = wdict).drop(['Loc'],axis =1)
loser = loser.rename(columns = ldict)
# changing other score to Oscore
winner.rename(columns ={'LScore':'OScore'},inplace = True)
loser.rename(columns ={'WScore':'OScore'},inplace = True)
loser.head()

,Season,OScore,TeamID,Score,FGM,FGA,FGM3,FGA3,FTM,FTA,OR,DR,Ast,TO,Stl,Blk,PF,loses,wins
0,2003,68,1328,62,22,53,2,10,16,22,10,22,8,18,9,2,20,1,0
1,2003,70,1393,63,24,67,6,24,9,20,20,25,7,12,8,6,16,1,0
2,2003,73,1437,61,22,73,3,26,14,23,31,22,9,12,2,5,23,1,0
3,2003,56,1457,50,18,49,6,22,8,15,17,20,9,19,4,3,23,1,0
4,2003,77,1208,71,24,62,6,16,17,27,21,15,12,10,7,1,14,1,0


In [90]:
# combine the loser and winners into one table, and get the averages for each team and each season and add the rank
games = winner.append(loser)
sums = games.groupby(['Season','TeamID']).sum()[['wins','loses']]
means = games.groupby(['Season','TeamID']).mean()[['Score','OScore','FGM','FGA','FGM3','FGA3','FTM',
                                                  'FTA','OR','DR','Ast','TO','Stl','Blk','PF']]
averages = sums.join(means)
averages['rank'] = rank.groupby(['Season','TeamID']).mean()['OrdinalRank']
averages.head()

wins  loses      Score     OScore        FGM        FGA  \
Season TeamID                                                            
2003   1102      12     16  57.250000  57.000000  19.142857  39.785714   
       1103      13     14  78.777778  78.148148  27.148148  55.851852   
       1104      17     11  69.285714  65.000000  24.035714  57.178571   
       1105       7     19  71.769231  76.653846  24.384615  61.615385   
       1106      13     15  63.607143  63.750000  23.428571  55.285714   

                   FGM3       FGA3        FTM        FTA         OR  \
Season TeamID                                                         
2003   1102    7.821429  20.821429  11.142857  17.107143   4.178571   
       1103    5.444444  16.074074  19.037037  25.851852   9.777778   
       1104    6.357143  19.857143  14.857143  20.928571  13.571429   
       1105    7.576923  20.769231  15.423077  21.846154  13.500000   
       1106    6.107143  17.642857  10.642857  16.464286  12.285714   

                      DR        Ast         TO       Stl       Blk         PF  \
Season TeamID                                                                   
2003   1102    16.821429  13.000000  11.428571  5.964286  1.785714  18.750000   
       1103    19.925926  15.222222  12.629630  7.259259  2.333333  19.851852   
       1104    23.928571  12.107143  13.285714  6.607143  3.785714  18.035714   
       1105    23.115385  14.538462  18.653846  9.307692  2.076923  20.230769   
       1106    23.857143  11.678571  17.035714  8.357143  3.142857  18.178571   

                     rank  
Season TeamID              
2003   1102    170.000000  
       1103    171.666667  
       1104     39.000000  
       1105    301.666667  
       1106    224.666667

In [91]:
tour_results

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,WFGM,WFGA,...,LFGA3,LFTM,LFTA,LOR,LDR,LAst,LTO,LStl,LBlk,LPF
0,2003,134,1421,92,1411,84,N,1,32,69,...,31,14,31,17,28,16,15,5,0,22
1,2003,136,1112,80,1436,51,N,0,31,66,...,16,7,7,8,26,12,17,10,3,15
2,2003,136,1113,84,1272,71,N,0,31,59,...,28,14,21,20,22,11,12,2,5,18
3,2003,136,1141,79,1166,73,N,0,29,53,...,17,12,17,14,17,20,21,6,6,21
4,2003,136,1143,76,1301,74,N,1,27,64,...,21,15,20,10,26,16,14,5,8,19
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1176,2021,148,1211,85,1425,66,N,0,33,66,...,15,14,19,7,20,9,9,7,0,13
1177,2021,148,1417,51,1276,49,N,0,21,54,...,11,6,11,8,24,12,14,5,3,11
1178,2021,152,1124,78,1222,59,N,0,29,55,...,19,11,16,13,12,10,10,4,5,10
1179,2021,152,1211,93,1417,90,N,1,37,63,...,17,14,21,7,24,21,9,4,1,16


In [92]:
# create our data to train our on resulting in data frame with TeamA, TeamB and if teamA won = 1
train = tour_results.copy()[['Season','WTeamID','LTeamID']]
train['pred'] =1
wins = train.iloc[:len(train)//2,:]
loses = train.iloc[len(train)//2:,:]

l = loses.copy().rename(columns = {'WTeamID':'L'})
l.rename(columns = {'LTeamID':'WTeamID'},inplace = True)
l.rename(columns = {'L':'LTeamID'},inplace = True)
l['pred'] = 0
train = pd.concat([wins,l])
train.rename(columns = {'WTeamID':"TeamA",'LTeamID':'TeamB'},inplace = True)
train

,Season,TeamA,TeamB,pred
0,2003,1421,1411,1
1,2003,1112,1436,1
2,2003,1113,1272,1
3,2003,1141,1166,1
4,2003,1143,1301,1
...,...,...,...,...
1176,2021,1425,1211,0
1177,2021,1276,1417,0
1178,2021,1222,1124,0
1179,2021,1417,1211,0


In [93]:
# adding a letter a or b to the beginning of each state
colA = {}
colB = {}
for cols in averages.columns:
    colA[cols] = "a" + cols
    colB[cols] = "b" + cols
#colA

In [97]:
# creating our full data set with teamA, TeamB, both their stats and the result.
data = averages.reset_index()
Adata = data.copy().rename(columns = colA)
Bdata = data.copy().rename(columns = colB)

train_a = train.merge(Adata,'left',left_on = ['Season','TeamA'],right_on=['Season','TeamID']).drop(['TeamID'],axis =1)
train_full = train_a.merge(Bdata,'left',left_on = ['Season','TeamB'],right_on=['Season','TeamID']).drop(['TeamID'],axis =1)

train_full.head()

,Season,TeamA,TeamB,pred,awins,aloses,aScore,aOScore,aFGM,aFGA,...,bFTM,bFTA,bOR,bDR,bAst,bTO,bStl,bBlk,bPF,brank
0,2003,1421,1411,1,13,16,71.206897,78.448276,24.379310,56.793103,...,17.400000,28.066667,13.166667,24.800000,14.200000,15.233333,6.433333,2.233333,18.300000,211.000000
1,2003,1112,1436,1,25,3,85.214286,70.250000,30.321429,65.714286,...,12.862069,19.551724,12.965517,25.724138,14.206897,14.068966,6.862069,2.965517,15.896552,133.666667
2,2003,1113,1272,1,18,11,75.965517,69.172414,27.206897,56.896552,...,14.965517,22.896552,14.068966,25.965517,16.620690,13.793103,7.379310,5.068966,18.758621,26.333333
3,2003,1141,1166,1,23,6,79.344828,73.241379,26.620690,52.689655,...,13.878788,20.030303,10.878788,23.181818,16.818182,13.363636,8.393939,4.454545,17.272727,17.333333
4,2003,1143,1301,1,21,8,74.482759,69.758621,27.344828,58.724138,...,15.766667,20.466667,9.733333,22.033333,14.666667,14.200000,7.766667,3.066667,18.666667,70.000000


In [98]:
train_full.tail()

,Season,TeamA,TeamB,pred,awins,aloses,aScore,aOScore,aFGM,aFGA,...,bFTM,bFTA,bOR,bDR,bAst,bTO,bStl,bBlk,bPF,brank
1176,2021,1425,1211,0,22,7,74.758621,65.103448,27.172414,58.172414,...,16.500000,22.730769,8.230769,27.269231,18.307692,11.923077,8.307692,2.846154,3.576923,1.074074
1177,2021,1276,1417,0,20,4,76.250000,65.375000,27.875000,57.625000,...,13.423077,18.615385,8.769231,24.192308,13.615385,10.884615,5.076923,2.653846,4.346154,41.901961
1178,2021,1222,1124,0,23,3,76.307692,58.307692,26.653846,60.807692,...,11.875000,17.041667,11.333333,22.375000,17.041667,12.166667,8.958333,3.750000,5.250000,2.296296
1179,2021,1417,1211,0,17,9,72.846154,68.500000,26.384615,57.115385,...,16.500000,22.730769,8.230769,27.269231,18.307692,11.923077,8.307692,2.846154,3.576923,1.074074
1180,2021,1211,1124,0,26,0,92.115385,69.115385,34.153846,62.000000,...,11.875000,17.041667,11.333333,22.375000,17.041667,12.166667,8.958333,3.750000,5.250000,2.296296


# Model Selection time

In [99]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

In [100]:
# creatinfg features
features = []
for i in colA.values():
    features.append(i)
for i in colB.values():
    features.append(i)
#features

In [102]:
# creating training and test sets
X_train, X_test, y_train, y_test = train_test_split(train_full,train_full['pred'], 
                                                    test_size=0.20)
X_train.head(10)

,Season,TeamA,TeamB,pred,awins,aloses,aScore,aOScore,aFGM,aFGA,...,bFTM,bFTA,bOR,bDR,bAst,bTO,bStl,bBlk,bPF,brank
88,2004,1280,1284,1,25,3,77.107143,66.285714,27.357143,58.500000,...,15.812500,21.187500,9.375000,24.000000,13.812500,13.406250,7.656250,2.625000,17.531250,157.666667
631,2012,1266,1196,0,25,7,75.937500,66.218750,26.500000,57.718750,...,13.484848,18.939394,11.454545,23.696970,15.000000,10.909091,6.484848,3.272727,15.848485,19.500000
688,2013,1143,1393,0,20,11,67.548387,64.387097,25.322581,56.774194,...,13.914286,20.600000,14.542857,24.200000,14.542857,12.628571,8.885714,6.200000,16.171429,15.636364
242,2006,1272,1133,1,30,3,80.909091,65.878788,28.060606,62.666667,...,13.766667,19.166667,12.600000,25.100000,16.533333,15.200000,9.200000,4.833333,18.733333,41.652174
68,2004,1181,1106,1,27,5,79.750000,64.968750,27.562500,58.468750,...,14.766667,22.466667,11.700000,22.766667,10.266667,17.200000,7.066667,3.133333,18.700000,251.000000
1071,2019,1153,1234,0,28,6,71.705882,62.205882,24.823529,57.441176,...,17.969697,24.272727,10.333333,25.393939,15.727273,12.151515,6.181818,3.272727,16.030303,39.984375
777,2014,1196,1163,0,32,2,70.735294,57.941176,24.382353,52.764706,...,15.970588,21.000000,10.088235,25.294118,12.588235,11.676471,6.970588,6.205882,18.264706,26.031250
1021,2018,1397,1260,0,25,8,74.212121,66.393939,25.242424,57.484848,...,12.343750,17.031250,6.156250,25.718750,15.437500,12.375000,6.656250,2.375000,14.000000,45.301587
437,2009,1257,1112,1,28,5,73.939394,61.606061,26.787879,59.424242,...,14.687500,20.000000,11.093750,22.750000,14.437500,12.593750,5.968750,3.062500,15.625000,56.000000
373,2008,1172,1458,1,25,6,77.322581,63.129032,28.645161,60.741935,...,14.545455,20.515152,11.606061,24.303030,12.787879,12.090909,6.303030,3.333333,14.333333,8.034483


In [103]:
clf = LogisticRegression(max_iter = 7000).fit(X_train[features], y_train)

In [104]:
predict = clf.predict(X_test[features])
probs = clf.predict_proba(X_test[features])

lloss = log_loss(y_test,probs)
accuracy = accuracy_score(y_test,predict)
lloss, accuracy

(0.2944123566670527, 0.8734177215189873)

In [105]:
#this log loss and accuracy seems very good espically for a random model
# lets see our predictions on the real data

In [106]:
# now to do the submission
sub = submission.copy()
sub['Season'] = sub['ID'].apply(lambda x: int(x.split('_')[0]))
sub['TeamIDA'] = sub['ID'].apply(lambda x: int(x.split('_')[1]))
sub['TeamIDB'] = sub['ID'].apply(lambda x: int(x.split('_')[2])) 

In [107]:
sub = sub.merge(Adata[Adata['Season'] == 2022],'left',left_on = ['Season','TeamIDA'],right_on=['Season','TeamID']).drop(['TeamID'],axis =1)
sub = sub.merge(Bdata[Bdata['Season'] == 2022],'left',left_on = ['Season','TeamIDB'],right_on=['Season','TeamID']).drop(['TeamID'],axis =1)
sub.head()

,ID,Pred,Season,TeamIDA,TeamIDB,awins,aloses,aScore,aOScore,aFGM,...,bFTM,bFTA,bOR,bDR,bAst,bTO,bStl,bBlk,bPF,brank
0,2022_1103_1104,0.5,2022,1103,1104,22,9,69.290323,64.16129,23.580645,...,15.375000,21.062500,11.625000,24.718750,14.625000,14.093750,7.000000,4.718750,18.593750,24.736842
1,2022_1103_1112,0.5,2022,1103,1112,22,9,69.290323,64.16129,23.580645,...,15.911765,21.558824,10.441176,28.352941,19.911765,12.941176,6.705882,5.705882,16.470588,2.350000
2,2022_1103_1116,0.5,2022,1103,1116,22,9,69.290323,64.16129,23.580645,...,17.303030,22.969697,9.696970,25.484848,14.090909,12.151515,7.727273,4.181818,16.848485,17.933333
3,2022_1103_1120,0.5,2022,1103,1120,22,9,69.290323,64.16129,23.580645,...,14.718750,20.093750,10.218750,25.718750,14.531250,11.656250,8.781250,7.843750,18.531250,5.333333
4,2022_1103_1124,0.5,2022,1103,1124,22,9,69.290323,64.16129,23.580645,...,12.125000,17.375000,10.968750,22.750000,15.843750,12.000000,8.812500,3.406250,15.843750,3.700000


In [111]:
sub

,ID,Pred,Season,TeamIDA,TeamIDB,awins,aloses,aScore,aOScore,aFGM,...,bFTM,bFTA,bOR,bDR,bAst,bTO,bStl,bBlk,bPF,brank
0,2022_1103_1104,0.5,2022,1103,1104,22,9,69.290323,64.161290,23.580645,...,15.375000,21.062500,11.625000,24.718750,14.625000,14.093750,7.000000,4.718750,18.593750,24.736842
1,2022_1103_1112,0.5,2022,1103,1112,22,9,69.290323,64.161290,23.580645,...,15.911765,21.558824,10.441176,28.352941,19.911765,12.941176,6.705882,5.705882,16.470588,2.350000
2,2022_1103_1116,0.5,2022,1103,1116,22,9,69.290323,64.161290,23.580645,...,17.303030,22.969697,9.696970,25.484848,14.090909,12.151515,7.727273,4.181818,16.848485,17.933333
3,2022_1103_1120,0.5,2022,1103,1120,22,9,69.290323,64.161290,23.580645,...,14.718750,20.093750,10.218750,25.718750,14.531250,11.656250,8.781250,7.843750,18.531250,5.333333
4,2022_1103_1124,0.5,2022,1103,1124,22,9,69.290323,64.161290,23.580645,...,12.125000,17.375000,10.968750,22.750000,15.843750,12.000000,8.812500,3.406250,15.843750,3.700000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2273,2022_1458_1461,0.5,2022,1458,1461,24,7,70.709677,66.548387,25.000000,...,14.656250,20.218750,6.906250,26.062500,10.875000,10.812500,3.781250,2.062500,15.281250,57.035088
2274,2022_1458_1463,0.5,2022,1458,1463,24,7,70.709677,66.548387,25.000000,...,13.931034,18.896552,7.517241,25.551724,11.827586,12.655172,6.000000,3.172414,17.586207,145.403509
2275,2022_1460_1461,0.5,2022,1460,1461,20,13,75.151515,71.606061,27.424242,...,14.656250,20.218750,6.906250,26.062500,10.875000,10.812500,3.781250,2.062500,15.281250,57.035088
2276,2022_1460_1463,0.5,2022,1460,1463,20,13,75.151515,71.606061,27.424242,...,13.931034,18.896552,7.517241,25.551724,11.827586,12.655172,6.000000,3.172414,17.586207,145.403509


In [114]:
X_test_real = sub[features]
probs_real = clf.predict_proba(X_test_real)
prediction_real = clf.predict(X_test_real)
real = []
for p in probs_real:
    real.append(p[1])
real[:10]

[0.0016498109910767999,
 0.0008457110189763687,
 0.0004082802092049497,
 9.277915509975006e-05,
 0.002050645546590798,
 0.0012241706130351104,
 0.001623702190122128,
 0.0002571262405473787,
 0.0012866047372277899,
 0.0002558703437170782]

In [115]:
sum(real)/len(real)

0.13562053369692803

something is very wrong now and the classifer is making an average prediction of .13, where as the average should be around 0.5.

I think the problem is that the model is very overtrained on the past tournament results. There is only 900 games to train on with my current split, where as in the submission section there is over 2000 games.

Lets try other models with a lot less features and see how that works

In [116]:
def win_percent(wins,loses):
    return wins/(wins+loses)
def point_diff(points,against):
    return points - against
def true_shooting(fga,fta,points):
    denominator = 2*(fga + (0.44*fta))
    return points/denominator
def total_rebounds(oreb,dreb):
    return oreb+dreb
def assist_turnover(ast,turn):
    return ast/turn

In [117]:
new_train = train_full.copy()
#wins
new_train['WinPercentA'] = new_train.apply(lambda x: win_percent(x['awins'],x['aloses']),axis = 1)
new_train['WinPercentB'] = new_train.apply(lambda x: win_percent(x['bwins'],x['bloses']),axis = 1)
new_train.drop(['awins','aloses','bwins','bloses'],axis = 1,inplace = True)

# true_shooting
new_train['TrueShootA'] = new_train.apply(lambda x: true_shooting(x['aFGA'],x['aFTA'],x['aScore']),axis = 1)
new_train['TrueShootB'] = new_train.apply(lambda x: true_shooting(x['bFGA'],x['bFTA'],x['bScore']),axis = 1)
new_train.drop(['aFGM','aFGA','aFTM','aFTA','aFGM3','aFGA3'],axis = 1,inplace = True)
new_train.drop(['bFGM','bFGA','bFTM','bFTA','bFGM3','bFGA3'],axis = 1,inplace = True)

# point diff
new_train['PointDiffA'] = new_train.apply(lambda x: point_diff(x['aScore'],x['aOScore']),axis = 1)
new_train['PointDiffB'] = new_train.apply(lambda x: point_diff(x['bScore'],x['bOScore']),axis = 1)
new_train.drop(['aScore','aOScore','bScore','bOScore'],axis = 1,inplace = True)

# total rebounds
new_train['TotalRebA'] = new_train.apply(lambda x: total_rebounds(x['aOR'],x['aDR']),axis = 1)
new_train['TotalRebB'] = new_train.apply(lambda x: total_rebounds(x['bOR'],x['bDR']),axis = 1)
new_train.drop(['aOR','aDR','bOR','bDR'],axis = 1,inplace = True)

# assist to turnover
new_train['ast-TurnA'] = new_train.apply(lambda x: assist_turnover(x['aAst'],x['aTO']),axis = 1)
new_train['ast-TurnB'] = new_train.apply(lambda x: assist_turnover(x['bAst'],x['bTO']),axis = 1)
new_train.drop(['aAst','aTO','bAst','bTO'],axis = 1,inplace = True)

# get rid of fouls
new_train.drop(['aPF','bPF'],axis = 1,inplace = True)
new_train.head()

,Season,TeamA,TeamB,pred,aStl,aBlk,arank,bStl,bBlk,brank,WinPercentA,WinPercentB,TrueShootA,TrueShootB,PointDiffA,PointDiffB,TotalRebA,TotalRebB,ast-TurnA,ast-TurnB
0,2003,1421,1411,1,7.068966,3.000000,230.000000,6.433333,2.233333,211.000000,0.448276,0.600000,0.539424,0.538334,-7.241379,1.966667,35.448276,37.966667,0.804255,0.932166
1,2003,1112,1436,1,8.464286,4.214286,1.666667,6.862069,2.965517,133.666667,0.892857,0.655172,0.555400,0.525815,14.964286,4.655172,42.821429,38.689655,1.193237,1.009804
2,2003,1113,1272,1,5.206897,4.241379,38.666667,7.379310,5.068966,26.333333,0.620690,0.793103,0.555080,0.531700,6.793103,8.689655,37.000000,40.034483,1.110837,1.205000
3,2003,1141,1166,1,7.103448,4.000000,57.666667,8.393939,4.454545,17.333333,0.793103,0.878788,0.622161,0.597895,6.103448,14.909091,33.862069,34.060606,0.856333,1.258503
4,2003,1143,1301,1,6.551724,2.793103,34.666667,7.766667,3.066667,70.000000,0.724138,0.600000,0.553267,0.580699,4.724138,4.400000,35.620690,31.766667,1.128954,1.032864


In [118]:
# now lets get the differences in stats for each match up
diffs = new_train.copy()
# win percent
diffs['WinPercent'] = diffs['WinPercentA'] - diffs['WinPercentB']
diffs.drop(['WinPercentA','WinPercentB'],axis = 1,inplace = True)
# point diff
diffs['PointDiff'] = diffs['PointDiffA'] - diffs['PointDiffB']
diffs.drop(['PointDiffA','PointDiffB'],axis = 1,inplace = True)
# true Shooting
diffs['TrueShooting'] = diffs['TrueShootA'] - diffs['TrueShootB']
diffs.drop(['TrueShootA','TrueShootB'],axis = 1,inplace = True)
# rebounds
diffs['TotalReb'] = diffs['TotalRebA'] - diffs['TotalRebB']
diffs.drop(['TotalRebA','TotalRebB'],axis = 1,inplace = True)
# assist to turnover
diffs['Ast-Turn'] = diffs['ast-TurnA'] - diffs['ast-TurnB']
diffs.drop(['ast-TurnA','ast-TurnB'],axis = 1,inplace = True)
# blocks
diffs['Blocks'] = diffs['aBlk'] - diffs['bBlk']
diffs.drop(['aBlk','bBlk'],axis = 1,inplace = True)
# steals
diffs['Steals'] = diffs['aStl'] - diffs['bStl']
diffs.drop(['aStl','bStl'],axis = 1,inplace = True)
# rank
diffs['Rank'] = diffs['arank'] - diffs['brank']
diffs.drop(['arank','brank'],axis = 1,inplace = True)
diffs.head()

,Season,TeamA,TeamB,pred,WinPercent,PointDiff,TrueShooting,TotalReb,Ast-Turn,Blocks,Steals,Rank
0,2003,1421,1411,1,-0.151724,-9.208046,0.001090,-2.518391,-0.127911,0.766667,0.635632,19.000000
1,2003,1112,1436,1,0.237685,10.309113,0.029586,4.131773,0.183433,1.248768,1.602217,-132.000000
2,2003,1113,1272,1,-0.172414,-1.896552,0.023379,-3.034483,-0.094163,-0.827586,-2.172414,12.333333
3,2003,1141,1166,1,-0.085684,-8.805643,0.024266,-0.198537,-0.402171,-0.454545,-1.290491,40.333333
4,2003,1143,1301,1,0.124138,0.324138,-0.027432,3.854023,0.096090,-0.273563,-1.214943,-35.333333


In [119]:
features =['WinPercent','PointDiff','TrueShooting','TotalReb','Ast-Turn','Blocks','Steals','Rank']

In [120]:
X_train, X_test, y_train, y_test = train_test_split(diffs,diffs['pred'], 
                                                    test_size=0.20)
X_train.head()

,Season,TeamA,TeamB,pred,WinPercent,PointDiff,TrueShooting,TotalReb,Ast-Turn,Blocks,Steals,Rank
409,2009,1281,1165,1,0.133874,6.405680,-0.017365,1.660243,0.301533,-0.078093,3.800203,-76.600000
611,2012,1388,1345,0,0.196970,4.751515,0.037396,3.169697,-0.267920,-0.309091,0.254545,36.500000
512,2011,1155,1412,1,-0.077083,0.745833,-0.000724,-1.093750,-0.146917,1.741667,2.697917,0.250000
816,2015,1326,1112,0,-0.214795,-4.399287,-0.004980,-1.386809,0.096895,1.502674,0.611408,17.516667
777,2014,1196,1163,0,0.176471,4.058824,-0.001799,0.294118,0.101135,-3.352941,0.235294,-23.281250


In [121]:
lr2 = LogisticRegression(max_iter = 5000).fit(X_train[features], y_train)

In [122]:
predict = lr2.predict(X_test[features])
probs = lr2.predict_proba(X_test[features])

lloss = log_loss(y_test,probs)
accuracy = accuracy_score(y_test,predict)
lloss, accuracy

(0.5210282998027791, 0.7257383966244726)

In [123]:
# now lets try a decison tree classifier and SVM
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC

In [124]:
dtc = DecisionTreeClassifier().fit(X_train[features],y_train)
predict = dtc.predict(X_test[features])
probs = dtc.predict_proba(X_test[features])

lloss = log_loss(y_test,probs)
accuracy = accuracy_score(y_test,predict)
lloss, accuracy

(12.24159163363923, 0.6455696202531646)

In [125]:
dtc.tree_

In [126]:
svc = SVC(probability=True).fit(X_train[features],y_train)
predict = svc.predict(X_test[features])
probs = svc.predict_proba(X_test[features])

lloss = log_loss(y_test,probs)
accuracy = accuracy_score(y_test,predict)
lloss, accuracy

(0.5499650094433071, 0.729957805907173)

In [127]:
#probs

In [128]:
sub2 = submission.copy()
sub2['Season'] = sub['ID'].apply(lambda x: int(x.split('_')[0]))
sub2['TeamIDA'] = sub['ID'].apply(lambda x: int(x.split('_')[1]))
sub2['TeamIDB'] = sub['ID'].apply(lambda x: int(x.split('_')[2])) 
sub2 = sub2.merge(Adata[Adata['Season'] == 2022],'left',left_on = ['Season','TeamIDA'],right_on=['Season','TeamID']).drop(['TeamID'],axis =1)
sub2 = sub2.merge(Bdata[Bdata['Season'] == 2022],'left',left_on = ['Season','TeamIDB'],right_on=['Season','TeamID']).drop(['TeamID'],axis =1)
sub2.head()

,ID,Pred,Season,TeamIDA,TeamIDB,awins,aloses,aScore,aOScore,aFGM,...,bFTM,bFTA,bOR,bDR,bAst,bTO,bStl,bBlk,bPF,brank
0,2022_1103_1104,0.5,2022,1103,1104,22,9,69.290323,64.16129,23.580645,...,15.375000,21.062500,11.625000,24.718750,14.625000,14.093750,7.000000,4.718750,18.593750,24.736842
1,2022_1103_1112,0.5,2022,1103,1112,22,9,69.290323,64.16129,23.580645,...,15.911765,21.558824,10.441176,28.352941,19.911765,12.941176,6.705882,5.705882,16.470588,2.350000
2,2022_1103_1116,0.5,2022,1103,1116,22,9,69.290323,64.16129,23.580645,...,17.303030,22.969697,9.696970,25.484848,14.090909,12.151515,7.727273,4.181818,16.848485,17.933333
3,2022_1103_1120,0.5,2022,1103,1120,22,9,69.290323,64.16129,23.580645,...,14.718750,20.093750,10.218750,25.718750,14.531250,11.656250,8.781250,7.843750,18.531250,5.333333
4,2022_1103_1124,0.5,2022,1103,1124,22,9,69.290323,64.16129,23.580645,...,12.125000,17.375000,10.968750,22.750000,15.843750,12.000000,8.812500,3.406250,15.843750,3.700000


In [129]:
new_train = sub2.copy()
#wins
new_train['WinPercentA'] = new_train.apply(lambda x: win_percent(x['awins'],x['aloses']),axis = 1)
new_train['WinPercentB'] = new_train.apply(lambda x: win_percent(x['bwins'],x['bloses']),axis = 1)
new_train.drop(['awins','aloses','bwins','bloses'],axis = 1,inplace = True)

# true_shooting
new_train['TrueShootA'] = new_train.apply(lambda x: true_shooting(x['aFGA'],x['aFTA'],x['aScore']),axis = 1)
new_train['TrueShootB'] = new_train.apply(lambda x: true_shooting(x['bFGA'],x['bFTA'],x['bScore']),axis = 1)
new_train.drop(['aFGM','aFGA','aFTM','aFTA','aFGM3','aFGA3'],axis = 1,inplace = True)
new_train.drop(['bFGM','bFGA','bFTM','bFTA','bFGM3','bFGA3'],axis = 1,inplace = True)

# point diff
new_train['PointDiffA'] = new_train.apply(lambda x: point_diff(x['aScore'],x['aOScore']),axis = 1)
new_train['PointDiffB'] = new_train.apply(lambda x: point_diff(x['bScore'],x['bOScore']),axis = 1)
new_train.drop(['aScore','aOScore','bScore','bOScore'],axis = 1,inplace = True)

# total rebounds
new_train['TotalRebA'] = new_train.apply(lambda x: total_rebounds(x['aOR'],x['aDR']),axis = 1)
new_train['TotalRebB'] = new_train.apply(lambda x: total_rebounds(x['bOR'],x['bDR']),axis = 1)
new_train.drop(['aOR','aDR','bOR','bDR'],axis = 1,inplace = True)

# assist to turnover
new_train['ast-TurnA'] = new_train.apply(lambda x: assist_turnover(x['aAst'],x['aTO']),axis = 1)
new_train['ast-TurnB'] = new_train.apply(lambda x: assist_turnover(x['bAst'],x['bTO']),axis = 1)
new_train.drop(['aAst','aTO','bAst','bTO'],axis = 1,inplace = True)

# get rid of fouls
new_train.drop(['aPF','bPF'],axis = 1,inplace = True)
new_train.head()

# now lets get the differences in stats for each match up
diffs = new_train.copy()
# win percent
diffs['WinPercent'] = diffs['WinPercentA'] - diffs['WinPercentB']
diffs.drop(['WinPercentA','WinPercentB'],axis = 1,inplace = True)
# point diff
diffs['PointDiff'] = diffs['PointDiffA'] - diffs['PointDiffB']
diffs.drop(['PointDiffA','PointDiffB'],axis = 1,inplace = True)
# true Shooting
diffs['TrueShooting'] = diffs['TrueShootA'] - diffs['TrueShootB']
diffs.drop(['TrueShootA','TrueShootB'],axis = 1,inplace = True)
# rebounds
diffs['TotalReb'] = diffs['TotalRebA'] - diffs['TotalRebB']
diffs.drop(['TotalRebA','TotalRebB'],axis = 1,inplace = True)
# assist to turnover
diffs['Ast-Turn'] = diffs['ast-TurnA'] - diffs['ast-TurnB']
diffs.drop(['ast-TurnA','ast-TurnB'],axis = 1,inplace = True)
# blocks
diffs['Blocks'] = diffs['aBlk'] - diffs['bBlk']
diffs.drop(['aBlk','bBlk'],axis = 1,inplace = True)
# steals
diffs['Steals'] = diffs['aStl'] - diffs['bStl']
diffs.drop(['aStl','bStl'],axis = 1,inplace = True)
# rank
diffs['Rank'] = diffs['arank'] - diffs['brank']
diffs.drop(['arank','brank'],axis = 1,inplace = True)
diffs

,ID,Pred,Season,TeamIDA,TeamIDB,WinPercent,PointDiff,TrueShooting,TotalReb,Ast-Turn,Blocks,Steals,Rank
0,2022_1103_1104,0.5,2022,1103,1104,0.115927,1.566532,0.011479,-4.988911,0.024806,-1.621976,-1.516129,112.298246
1,2022_1103_1112,0.5,2022,1103,1112,-0.202087,-11.900380,-0.029909,-7.439279,-0.476136,-2.609108,-1.222011,134.685088
2,2022_1103_1116,0.5,2022,1103,1116,-0.047898,-3.264907,0.022655,-3.826979,-0.097101,-1.085044,-2.243402,119.101754
3,2022_1103_1120,0.5,2022,1103,1120,-0.134073,-6.558468,0.023073,-4.582661,-0.184149,-4.746976,-3.297379,131.701754
4,2022_1103_1124,0.5,2022,1103,1124,-0.102823,-7.745968,0.008452,-2.363911,-0.257812,-0.309476,-3.328629,133.335088
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2273,2022_1458_1461,0.5,2022,1458,1461,0.024194,-2.026210,-0.039151,-0.807460,0.300733,0.743952,1.509073,-36.018421
2274,2022_1458_1463,0.5,2022,1458,1463,0.153504,2.126808,-0.016403,-0.907675,0.371909,-0.365962,-0.709677,-124.386842
2275,2022_1460_1461,0.5,2022,1460,1461,-0.143939,-2.642045,-0.003757,-1.150568,0.128883,0.846591,1.976326,127.421053
2276,2022_1460_1463,0.5,2022,1460,1463,-0.014629,1.510972,0.018991,-1.250784,0.200058,-0.263323,-0.242424,39.052632


In [130]:
predict = lr2.predict(diffs[features])
probs = lr2.predict_proba(diffs[features])

In [133]:
real = []
for p in probs:
    real.append(p[1])
real[:10]

[0.1322842298712436,
 0.04699431525908171,
 0.09273074935223972,
 0.044671407439075675,
 0.054534870303321444,
 0.13327505315363836,
 0.7435965855689767,
 0.24378106052089982,
 0.45530715714167547,
 0.1242204895309725]

In [134]:
sum(real)/len(real)

0.5238210806833551

this number seems a lot more resonable

In [135]:
submission['Pred'] = real
submission.head()

,ID,Pred
0,2022_1103_1104,0.132284
1,2022_1103_1112,0.046994
2,2022_1103_1116,0.092731
3,2022_1103_1120,0.044671
4,2022_1103_1124,0.054535


In [136]:
submission.to_csv('Submission',index = False)

In [137]:
submission

,ID,Pred
0,2022_1103_1104,0.132284
1,2022_1103_1112,0.046994
2,2022_1103_1116,0.092731
3,2022_1103_1120,0.044671
4,2022_1103_1124,0.054535
...,...,...
2273,2022_1458_1461,0.654083
2274,2022_1458_1463,0.892576
2275,2022_1460_1461,0.119156
2276,2022_1460_1463,0.372815


In [138]:
sub

,ID,Pred,Season,TeamIDA,TeamIDB,awins,aloses,aScore,aOScore,aFGM,...,bFTM,bFTA,bOR,bDR,bAst,bTO,bStl,bBlk,bPF,brank
0,2022_1103_1104,0.5,2022,1103,1104,22,9,69.290323,64.161290,23.580645,...,15.375000,21.062500,11.625000,24.718750,14.625000,14.093750,7.000000,4.718750,18.593750,24.736842
1,2022_1103_1112,0.5,2022,1103,1112,22,9,69.290323,64.161290,23.580645,...,15.911765,21.558824,10.441176,28.352941,19.911765,12.941176,6.705882,5.705882,16.470588,2.350000
2,2022_1103_1116,0.5,2022,1103,1116,22,9,69.290323,64.161290,23.580645,...,17.303030,22.969697,9.696970,25.484848,14.090909,12.151515,7.727273,4.181818,16.848485,17.933333
3,2022_1103_1120,0.5,2022,1103,1120,22,9,69.290323,64.161290,23.580645,...,14.718750,20.093750,10.218750,25.718750,14.531250,11.656250,8.781250,7.843750,18.531250,5.333333
4,2022_1103_1124,0.5,2022,1103,1124,22,9,69.290323,64.161290,23.580645,...,12.125000,17.375000,10.968750,22.750000,15.843750,12.000000,8.812500,3.406250,15.843750,3.700000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2273,2022_1458_1461,0.5,2022,1458,1461,24,7,70.709677,66.548387,25.000000,...,14.656250,20.218750,6.906250,26.062500,10.875000,10.812500,3.781250,2.062500,15.281250,57.035088
2274,2022_1458_1463,0.5,2022,1458,1463,24,7,70.709677,66.548387,25.000000,...,13.931034,18.896552,7.517241,25.551724,11.827586,12.655172,6.000000,3.172414,17.586207,145.403509
2275,2022_1460_1461,0.5,2022,1460,1461,20,13,75.151515,71.606061,27.424242,...,14.656250,20.218750,6.906250,26.062500,10.875000,10.812500,3.781250,2.062500,15.281250,57.035088
2276,2022_1460_1463,0.5,2022,1460,1463,20,13,75.151515,71.606061,27.424242,...,13.931034,18.896552,7.517241,25.551724,11.827586,12.655172,6.000000,3.172414,17.586207,145.403509


In [139]:
sub = submission.copy()
sub['Season'] = sub['ID'].apply(lambda x: int(x.split('_')[0]))
sub['TeamIDA'] = sub['ID'].apply(lambda x: int(x.split('_')[1]))
sub['TeamIDB'] = sub['ID'].apply(lambda x: int(x.split('_')[2])) 
name_mapping = sub.copy()
name_mapping.merge(names,left_on =['TeamIDA'],right_on=['TeamID']).drop(['FirstD1Season',
                                                                         'LastD1Season','TeamID'],axis = 1)

name_mapping
#name_mapping.merge(names,left_on =['TeamIDB'],right_on=['TeamID']).drop(['FirstD1Season',
 #                                                                        'LastD1Season','TeamID'],axis = 1)

,ID,Pred,Season,TeamIDA,TeamIDB
0,2022_1103_1104,0.132284,2022,1103,1104
1,2022_1103_1112,0.046994,2022,1103,1112
2,2022_1103_1116,0.092731,2022,1103,1116
3,2022_1103_1120,0.044671,2022,1103,1120
4,2022_1103_1124,0.054535,2022,1103,1124
...,...,...,...,...,...
2273,2022_1458_1461,0.654083,2022,1458,1461
2274,2022_1458_1463,0.892576,2022,1458,1463
2275,2022_1460_1461,0.119156,2022,1460,1461
2276,2022_1460_1463,0.372815,2022,1460,1463


In [140]:
sub = submission.copy()
sub['TeamIDA'] = sub['ID'].apply(lambda x: int(x.split('_')[1]))
sub['TeamIDB'] = sub['ID'].apply(lambda x: int(x.split('_')[2])) 

In [142]:
names_dict = names.drop(['FirstD1Season',"LastD1Season"], axis = 1).set_index('TeamID').to_dict()
real_names_dict = names_dict['TeamName']
real_names_dict

{1101: 'Abilene Chr',
 1102: 'Air Force',
 1103: 'Akron',
 1104: 'Alabama',
 1105: 'Alabama A&M',
 1106: 'Alabama St',
 1107: 'SUNY Albany',
 1108: 'Alcorn St',
 1109: 'Alliant Intl',
 1110: 'American Univ',
 1111: 'Appalachian St',
 1112: 'Arizona',
 1113: 'Arizona St',
 1114: 'Ark Little Rock',
 1115: 'Ark Pine Bluff',
 1116: 'Arkansas',
 1117: 'Arkansas St',
 1118: 'Armstrong St',
 1119: 'Army',
 1120: 'Auburn',
 1121: 'Augusta',
 1122: 'Austin Peay',
 1123: 'Ball St',
 1124: 'Baylor',
 1125: 'Belmont',
 1126: 'Bethune-Cookman',
 1127: 'Binghamton',
 1128: 'Birmingham So',
 1129: 'Boise St',
 1130: 'Boston College',
 1131: 'Boston Univ',
 1132: 'Bowling Green',
 1133: 'Bradley',
 1134: 'Brooklyn',
 1135: 'Brown',
 1136: 'Bryant',
 1137: 'Bucknell',
 1138: 'Buffalo',
 1139: 'Butler',
 1140: 'BYU',
 1141: 'C Michigan',
 1142: 'Cal Poly',
 1143: 'California',
 1144: 'Campbell',
 1145: 'Canisius',
 1146: 'Cent Arkansas',
 1147: 'Centenary',
 1148: 'Central Conn',
 1149: 'Charleston So',

In [143]:
sub.replace({'TeamIDA':real_names_dict},inplace = True)
sub.replace({'TeamIDB':real_names_dict},inplace = True)
sub.to_csv('excel',index = False)